In [1]:
spark.sql("CREATE SCHEMA IF NOT EXISTS bronze")

StatementMeta(, e0178098-21f4-49c6-a9e7-9dac3530b17e, 3, Finished, Available, Finished)

DataFrame[]

In [1]:
raw_path = "Files/raw-landing/nyc_taxi"

df_raw = spark.read.parquet(raw_path)

df_raw.printSchema()
df_raw.show(5)


StatementMeta(, c3695e46-77d3-4e10-8e7f-5ba5fd8dd87b, 3, Finished, Available, Finished)

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- cbd_congestion_fee: double (nullable = true)

+--------+--------------------+---------------------+---------------+------

In [2]:
from pyspark.sql.functions import current_timestamp, input_file_name

df_bronze = (
    df_raw
    .withColumn("processed_timestamp", current_timestamp())
    .withColumn("source_file", input_file_name())  # optional but very good
)

df_bronze.select("processed_timestamp", "source_file").show(5)


StatementMeta(, c3695e46-77d3-4e10-8e7f-5ba5fd8dd87b, 4, Finished, Available, Finished)

+--------------------+--------------------+
| processed_timestamp|         source_file|
+--------------------+--------------------+
|2026-01-19 04:11:...|abfss://9c8d15ad-...|
|2026-01-19 04:11:...|abfss://9c8d15ad-...|
|2026-01-19 04:11:...|abfss://9c8d15ad-...|
|2026-01-19 04:11:...|abfss://9c8d15ad-...|
|2026-01-19 04:11:...|abfss://9c8d15ad-...|
+--------------------+--------------------+
only showing top 5 rows



In [3]:
df_bronze.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("bronze.nyc_taxi_trips_bronze")


StatementMeta(, c3695e46-77d3-4e10-8e7f-5ba5fd8dd87b, 5, Finished, Available, Finished)

In [4]:
spark.sql("SELECT COUNT(*) FROM bronze.nyc_taxi_trips_bronze").show()


StatementMeta(, c3695e46-77d3-4e10-8e7f-5ba5fd8dd87b, 6, Finished, Available, Finished)

+--------+
|count(1)|
+--------+
|55990130|
+--------+

